### Датасет - https://www.kaggle.com/datasets/ayush1220/cifar10

In [1]:
!uv pip install --system ultralytics

Using Python 3.12.12 environment at: /usr
Resolved 49 packages in 403ms                                        
Prepared 2 packages in 107ms                                             
Installed 2 packages in 12ms                                
 + ultralytics==8.4.13
 + ultralytics-thop==2.0.18


In [2]:
import os
import pandas as pd

train_path = '/kaggle/input/cifar10/cifar10/train'

rows = []

for class_name in os.listdir(train_path):
    class_path = os.path.join(train_path, class_name)

    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        rows.append({"class_name": class_name, "image_path": img_path})

df = pd.DataFrame(rows)
df.head()

,class_name,image_path
0,airplane,/kaggle/input/cifar10/cifar10/train/airplane/0...
1,airplane,/kaggle/input/cifar10/cifar10/train/airplane/4...
2,airplane,/kaggle/input/cifar10/cifar10/train/airplane/2...
3,airplane,/kaggle/input/cifar10/cifar10/train/airplane/0...
4,airplane,/kaggle/input/cifar10/cifar10/train/airplane/2...


In [3]:
import os
import shutil
import random
import pandas as pd

df['used'] = False

# Создаем папку dataset
dataset_path = '/kaggle/working/dataset'
os.makedirs(dataset_path, exist_ok=True)

# 1. Копируем всю папку /content/cifar10/val в dataset
val_src = '/kaggle/input/cifar10/cifar10/test'
val_dst = os.path.join(dataset_path, 'val')
shutil.copytree(val_src, val_dst)

'/kaggle/working/dataset/val'

In [4]:
# 2. Создаем папку train в dataset
train_dst = os.path.join(dataset_path, 'train')
os.makedirs(train_dst, exist_ok=True)

# ИЗ 50 000 изображений в df БЕРЕМ СЛУЧАЙНЫЕ 500
selected_df = df.sample(n=500)

# Создаем папки классов и копируем изображения + помечаем used=True
for _, row in selected_df.iterrows():
    class_name = row['class_name']
    img_path = row['image_path']
    img_name = os.path.basename(img_path)

    class_dst_path = os.path.join(train_dst, class_name)
    os.makedirs(class_dst_path, exist_ok=True)

    dst_img = os.path.join(class_dst_path, img_name)
    shutil.copy2(img_path, dst_img)

    # ПОМЕЧАЕМ КАК ИСПОЛЬЗОВАННОЕ
    df.loc[df['image_path'] == img_path, 'used'] = True

print(f"Скопировано 500 случайных изображений из 50 000")
print("Все помечены used=True")
print("Распределение по классам:")
print(selected_df['class_name'].value_counts().sort_index())

Скопировано 500 случайных изображений из 50 000
Все помечены used=True
Распределение по классам:
class_name
airplane      56
automobile    49
bird          44
cat           56
deer          37
dog           54
frog          54
horse         46
ship          56
truck         48
Name: count, dtype: int64


In [5]:
epochs_value = 100
imgsz_value = 32
batch_value = 128

In [6]:
from ultralytics import YOLO

# Загрузка предобученной модели классификатора
model = YOLO('yolo11n-cls.pt')

# Обучение
results = model.train(
    data=dataset_path,
    epochs=epochs_value,
    imgsz=imgsz_value,
    batch=batch_value
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.13 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=32, int8=Fal

### DAL

In [ ]:
import numpy as np
from tqdm import tqdm

for iter in range(1, 10+1):
    print(f'Итерация {iter}')

    df['entropy'] = None

    best_model_path = os.path.join(results.save_dir, 'weights/best.pt')
    model = YOLO(best_model_path)  

    for idx in tqdm(df[df['used'] == False].index, desc='Вычисление entropy для неразмеченных примеров'):
        results = model(df.iloc[idx]['image_path'], verbose=False, imgsz=imgsz_value)

        # Получение названий классов и вероятностей
        # names = results[0].names
        probs = results[0].probs.data.cpu().numpy()

        # Расчет энтропии Шеннона
        entropy = -np.sum(probs * np.log2(probs + 1e-12))

        df.at[idx, 'entropy'] = entropy

    # Сортировка по убыванию энтропии
    count = 500  # сколько отобрать примеров с высоким entropy

    indexes_max_entropy = df[df['used'] == False].sort_values(by='entropy', ascending=False).index[:count]

    for idx in indexes_max_entropy:
        df.at[idx, 'used'] = True

    df['used'].value_counts()

    !rm -rf /kaggle/working/dataset/train.cache
    !rm -rf /kaggle/working/dataset/train

    # Базовая папка
    base_dir = '/kaggle/working/dataset'
    train_dir = os.path.join(base_dir, "train")

    # Создаём главную папку train, если её ещё нет
    os.makedirs(train_dir, exist_ok=True)

    # Берём только размеченные (used == True) строки
    used_df = df[df['used'] == True]

    print(f"Всего файлов для копирования: {len(used_df)}")

    used_df.to_csv(f'train_iter{iter}')

    # Проходим по каждой строке с прогресс-баром
    for idx, row in tqdm(used_df.iterrows(), total=len(used_df), desc="Копирование файлов"):
        class_name = row['class_name']
        src_path   = row['image_path']

        # Папка для текущего класса
        class_dir = os.path.join(train_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)

        # Имя файла берём из оригинального пути
        filename = os.path.basename(src_path)
        dst_path = os.path.join(class_dir, filename)

        # Копируем файл с сохранением метаданных (shutil.copy2)
        shutil.copy2(src_path, dst_path)

    print("\nКопирование завершено.")
    print(f"Структура создана в: {train_dir}")

    # Опционально: посмотреть, сколько файлов в каждой папке класса
    print("\nКоличество файлов по классам:")
    for class_name in sorted(used_df['class_name'].unique()):
        class_path = os.path.join(train_dir, class_name)
        if os.path.exists(class_path):
            count = len(os.listdir(class_path))
            print(f"  {class_name:12} → {count:4} файлов")
        else:
            print(f"  {class_name:12} → 0 файлов")

    # Загрузка предобученной модели классификатора
    model = YOLO(best_model_path)
    
    # Обучение
    results = model.train(
        data=dataset_path,
        epochs=epochs_value,
        imgsz=imgsz_value,
        batch=batch_value
    )

Итерация 1


Вычисление entropy для неразмеченных примеров: 100%|██████████| 49500/49500 [10:58<00:00, 75.14it/s]


Всего файлов для копирования: 1000


Копирование файлов: 100%|██████████| 1000/1000 [00:01<00:00, 797.45it/s]



Копирование завершено.
Структура создана в: /kaggle/working/dataset/train

Количество файлов по классам:
  airplane     →   92 файлов
  automobile   →   96 файлов
  bird         →  106 файлов
  cat          →  147 файлов
  deer         →  101 файлов
  dog          →  115 файлов
  frog         →   98 файлов
  horse        →   84 файлов
  ship         →   83 файлов
  truck        →   78 файлов
Ultralytics 8.4.13 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fractio

...

Вычисление entropy для неразмеченных примеров: 100%|██████████| 48000/48000 [04:33<00:00, 175.22it/s]


Всего файлов для копирования: 2500


Копирование файлов: 100%|██████████| 2500/2500 [00:02<00:00, 947.34it/s]



Копирование завершено.
Структура создана в: /kaggle/working/dataset/train

Количество файлов по классам:
  airplane     →  221 файлов
  automobile   →  211 файлов
  bird         →  322 файлов
  cat          →  374 файлов
  deer         →  252 файлов
  dog          →  281 файлов
  frog         →  186 файлов
  horse        →  234 файлов
  ship         →  193 файлов
  truck        →  226 файлов
Ultralytics 8.4.13 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fractio

Вычисление entropy для неразмеченных примеров:  85%|████████▌ | 40515/47500 [03:50<00:38, 181.66it/s]